# SQLite

___

## Dump

In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('track.db')
cur = conn.cursor()

In [89]:
cur.executescript("""
    DROP TABLE Artist;
    DROP TABLE Genre;
    DROP TABLE Album;
    DROP TABLE Track;

    CREATE TABLE Artist(
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        name TEXT
    );
    
    CREATE TABLE Genre(
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        name TEXT
    );
    
    CREATE TABLE Album(
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        title TEXT,
        artist_id INTEGER
    );
    
    CREATE TABLE Track(
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        title TEXT,
        length INTEGER,
        rating INTEGER,
        count INTEGER,
        album_id INTEGER,
        genre_id INTEGER
    );
""")

cur.execute("INSERT into Artist (name) VALUES ('Led Zepplin')")
cur.execute("INSERT INTO Artist (name) VALUES ('AC/DC')")
cur.execute("INSERT INTO Genre (name) VALUES ('Rock')")
cur.execute("INSERT INTO Genre (name) VALUES ('Metal')")
cur.execute("INSERT INTO Album (title, artist_id) VALUES ('Who Made Who', 2)")
cur.execute("INSERT INTO Album (title, artist_id) VALUES ('IV', 1)")
cur.execute("INSERT INTO Track (title, rating, length, count, album_id, genre_id) VALUES ('Black Dog', 5, 297, 0, 2, 1)")
cur.execute("INSERT INTO Track (title, rating, length, count, album_id, genre_id) VALUES ('Stairway', 5, 482, 0, 2, 1)")
cur.execute("INSERT INTO Track (title, rating, length, count, album_id, genre_id) VALUES ('About to Rock', 5, 313, 0, 1, 2)")
cur.execute("INSERT INTO Track (title, rating, length, count, album_id, genre_id) VALUES ('Who Made Who', 5, 207, 0, 1, 2)")

conn.commit()

In [90]:
cur.execute("SELECT Album.title, Artist.name from Album JOIN Artist ON Album.artist_id = Artist.id")
for row in cur.fetchall():
    print(row)

('Who Made Who', 'AC/DC')
('IV', 'Led Zepplin')


In [91]:
cur.execute("SELECT * from Artist cross join Album")
for row in cur.fetchall():
    print(row)

(1, 'Led Zepplin', 1, 'Who Made Who', 2)
(1, 'Led Zepplin', 2, 'IV', 1)
(2, 'AC/DC', 1, 'Who Made Who', 2)
(2, 'AC/DC', 2, 'IV', 1)


In [92]:
cur.execute("SELECT * FROM Artist join Album ON Artist.id = Album.artist_id")
for row in cur.fetchall():
    print(row)

(2, 'AC/DC', 1, 'Who Made Who', 2)
(1, 'Led Zepplin', 2, 'IV', 1)


In [93]:
cur.execute("SELECT Artist.name, Album.title FROM Artist join Album ON Artist.id = Album.artist_id")
for row in cur.fetchall():
    print(row)

('AC/DC', 'Who Made Who')
('Led Zepplin', 'IV')


In [94]:
cur.execute("SELECT * from Album JOIN Track ON Album.id = Track.album_id")
for row in cur.fetchall():
    print(row)

(2, 'IV', 1, 1, 'Black Dog', 297, 5, 0, 2, 1)
(2, 'IV', 1, 2, 'Stairway', 482, 5, 0, 2, 1)
(1, 'Who Made Who', 2, 3, 'About to Rock', 313, 5, 0, 1, 2)
(1, 'Who Made Who', 2, 4, 'Who Made Who', 207, 5, 0, 1, 2)


In [95]:
cur.execute("SELECT Track.title, Album.title, Genre.name, Artist.name FROM Track JOIN Album JOIN Genre JOIN Artist on Track.album_id = Album.id and Track.genre_id = Genre.id and Album.artist_id = Artist.id")
for row in cur.fetchall():
    print(row)

('Black Dog', 'IV', 'Rock', 'Led Zepplin')
('Stairway', 'IV', 'Rock', 'Led Zepplin')
('About to Rock', 'Who Made Who', 'Metal', 'AC/DC')
('Who Made Who', 'Who Made Who', 'Metal', 'AC/DC')


In [97]:
con = sqlite3.connect('track.db')
with open('dump.sql', 'w') as f:
    for line in con.iterdump():
        f.write('%s\n' % line)

dump: 백업 기능.

dump할 db에 connect한 다음 open의 argument로 백업할 파일명을 넣어준다.

In [98]:
data = None
with open('dump.sql', 'r') as f:
    data = f.read()

In [99]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
cur.executescript(data)

위 명령 실행시 dump로 저장된 sql이 전부 실행된다.

___

## Join

In [100]:
conn = sqlite3.connect('track.db')
cur = conn.cursor()

In [101]:
cur.execute("INSERT INTO Track (title) values ('아무거나')")

In [102]:
cur.execute("SELECT * FROM Track")
cur.fetchall()

[(1, 'Black Dog', 297, 5, 0, 2, 1),
 (2, 'Stairway', 482, 5, 0, 2, 1),
 (3, 'About to Rock', 313, 5, 0, 1, 2),
 (4, 'Who Made Who', 207, 5, 0, 1, 2),
 (5, '아무거나', None, None, None, None, None)]

In [103]:
cur.execute("SELECT * FROM Track LEFT JOIN Album ON Album.id = Track.album_id")
cur.fetchall()

[(1, 'Black Dog', 297, 5, 0, 2, 1, 2, 'IV', 1),
 (2, 'Stairway', 482, 5, 0, 2, 1, 2, 'IV', 1),
 (3, 'About to Rock', 313, 5, 0, 1, 2, 1, 'Who Made Who', 2),
 (4, 'Who Made Who', 207, 5, 0, 1, 2, 1, 'Who Made Who', 2),
 (5, '아무거나', None, None, None, None, None, None, None, None)]

In [104]:
cur.execute("SELECT * FROM Track RIGHT JOIN Album ON Album.id = Track.album_id")
cur.fetchall()

OperationalError: RIGHT and FULL OUTER JOINs are not currently supported

SQLite에서 Right join은 지원되지 않음.

In [88]:
cur.execute("SELECT * FROM Album LEFT JOIN Track ON Album.id = Track.album_id")
cur.fetchall()

[(1, 'Who Made Who', 2, 3, 'About to Rock', 313, 5, 0, 1, 2),
 (1, 'Who Made Who', 2, 4, 'Who Made Who', 207, 5, 0, 1, 2),
 (2, 'IV', 1, 1, 'Black Dog', 297, 5, 0, 2, 1),
 (2, 'IV', 1, 2, 'Stairway', 482, 5, 0, 2, 1)]

같은 Left join이어도 인자로 들어가는 테이블 이름 순서만 바뀌어도 결과가 다르게 나온다.

where, join 차이: where가 편하지만 시간, overhead 문제로 join 권장.